<a href="https://www.kaggle.com/code/sawsanshakir/stress-detection-using-pyspark?scriptVersionId=118036076" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/soaicommunitydatathon23-stress-detection/sample.csv
/kaggle/input/soaicommunitydatathon23-stress-detection/train.csv
/kaggle/input/soaicommunitydatathon23-stress-detection/test.csv


In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 12.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=de5438b9fdd5ffa33d89f9e97084710f426219d4389536a36c3edaf2c6af5dc2
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql.types import StructType,StructField,DoubleType
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [4]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
sc =SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

data = sqlContext.read.format('com.databricks.spark.csv').option("escape", "\"").options(header='true', inferschema='true').load('/kaggle/input/soaicommunitydatathon23-stress-detection/train.csv')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/02 14:47:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/opt/conda/lib/python3.7/site-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [5]:
data.show(2)

23/02/02 14:47:57 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/02/02 14:47:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, subreddit, post_id, sentence_range, text, id, label, confidence, social_timestamp, social_karma, syntax_ari, lex_liwc_WC, lex_liwc_Analytic, lex_liwc_Clout, lex_liwc_Authentic, lex_liwc_Tone, lex_liwc_WPS, lex_liwc_Sixltr, lex_liwc_Dic, lex_liwc_function, lex_liwc_pronoun, lex_liwc_ppron, lex_liwc_i, lex_liwc_we, lex_liwc_you, lex_liwc_shehe, lex_liwc_they, lex_liwc_ipron, lex_liwc_article, lex_liwc_prep, lex_liwc_auxverb, lex_liwc_adverb, lex_liwc_conj, lex_liwc_negate, lex_liwc_verb, lex_liwc_adj, lex_liwc_compare, lex_liwc_interrog, lex_liwc_number, lex_liwc_quant, lex_liwc_affect, lex_liwc_posemo, lex_liwc_negemo, lex_liwc_anx, lex_liwc_anger, lex_liwc_sad, lex_liwc_social, lex_liwc_family, lex_liwc_frien

In [6]:
##col_list = ['subreddit','sentence_range','id','confidence','social_timestamp','social_karma','syntax_ari','lex_liwc_WC','lex_liwc_Analytic','lex_liwc_Clout','lex_liwc_Authentic','lex_liwc_Tone','lex_liwc_WPS','lex_liwc_Sixltr','lex_liwc_Dic','lex_liwc_function','lex_liwc_pronoun','lex_liwc_ppron','lex_liwc_i','lex_liwc_we','lex_liwc_you','lex_liwc_shehe','lex_liwc_they','lex_liwc_ipron','lex_liwc_article','lex_liwc_prep','lex_liwc_auxverb','lex_liwc_adverb','lex_liwc_conj','lex_liwc_negate','lex_liwc_verb','lex_liwc_adj','lex_liwc_compare','lex_liwc_interrog','lex_liwc_number','lex_liwc_quant','lex_liwc_affect','lex_liwc_posemo','lex_liwc_negemo','lex_liwc_anx','lex_liwc_anger','lex_liwc_sad','lex_liwc_social','lex_liwc_family','lex_liwc_friend','lex_liwc_female','lex_liwc_male','lex_liwc_cogproc','lex_liwc_insight','lex_liwc_cause','lex_liwc_discrep','lex_liwc_tentat','lex_liwc_certain','lex_liwc_differ','lex_liwc_percept','lex_liwc_see','lex_liwc_hear','lex_liwc_feel','lex_liwc_bio','lex_liwc_body','lex_liwc_health','lex_liwc_sexual','lex_liwc_ingest','lex_liwc_drives','lex_liwc_affiliation','lex_liwc_achieve','lex_liwc_power','lex_liwc_reward','lex_liwc_risk','lex_liwc_focuspast','lex_liwc_focuspresent','lex_liwc_focusfuture','lex_liwc_relativ','lex_liwc_motion','lex_liwc_space','lex_liwc_time','lex_liwc_work','lex_liwc_leisure','lex_liwc_home','lex_liwc_money','lex_liwc_relig','lex_liwc_death','lex_liwc_informal','lex_liwc_swear','lex_liwc_netspeak','lex_liwc_assent','lex_liwc_nonflu','lex_liwc_filler','lex_liwc_AllPunc','lex_liwc_Period','lex_liwc_Comma','lex_liwc_Colon','lex_liwc_SemiC','lex_liwc_QMark','lex_liwc_Exclam','lex_liwc_Dash','lex_liwc_Quote','lex_liwc_Apostro','lex_liwc_Parenth','lex_liwc_OtherP','lex_dal_max_pleasantness','lex_dal_max_activation','lex_dal_max_imagery','lex_dal_min_pleasantness','lex_dal_min_activation','lex_dal_min_imagery','lex_dal_avg_activation','lex_dal_avg_imagery','lex_dal_avg_pleasantness','social_upvote_ratio','social_num_comments','syntax_fk_grade','sentiment']

col_list = ['post_id0', 'text', 'label']
data = data.select([column for column in data.columns if column in col_list])

In [7]:
data.show(10)

23/02/02 14:47:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv
+--------+--------------------+-----+
|post_id0|                text|label|
+--------+--------------------+-----+
|    2092|Hello, You are in...|    0|
|    1567|I want to know ab...|    0|
|    1550|Have any of you r...|    0|
|    2785|Whenever I have a...|    1|
|    1936|Throwaway because...|    1|
|    1125|But I really do e...|    1|
|     945|We are 21 now, bo...|    0|
|    1811|I have started wo...|    0|
|    2671|  ---   **tl;dr**...|    0|
|    1463|I just feel so na...|    1|
+--------+--------------------+-----+
only showing top 10 rows



Spark Machine Learning Pipelines API is similar to Scikit-Learn. Our pipeline includes three steps:

-regexTokenizer: Tokenization (with Regular Expression)

-stopwordsRemover: Remove Stop Words

-countVectors: Count vectors (“document-term vectors”)

In [8]:
from pyspark.sql import SQLContext
from pyspark import SparkContext

sc_1 =SparkContext.getOrCreate()
sqlContext_1 = SQLContext(sc_1)

data_test = sqlContext.read.format('com.databricks.spark.csv').option("escape", "\"").options(header='true', inferschema='true').load('/kaggle/input/soaicommunitydatathon23-stress-detection/test.csv')

In [9]:
data_test.show(1)

23/02/02 14:48:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, subreddit, post_id, sentence_range, text, id, confidence, social_timestamp, social_karma, syntax_ari, lex_liwc_WC, lex_liwc_Analytic, lex_liwc_Clout, lex_liwc_Authentic, lex_liwc_Tone, lex_liwc_WPS, lex_liwc_Sixltr, lex_liwc_Dic, lex_liwc_function, lex_liwc_pronoun, lex_liwc_ppron, lex_liwc_i, lex_liwc_we, lex_liwc_you, lex_liwc_shehe, lex_liwc_they, lex_liwc_ipron, lex_liwc_article, lex_liwc_prep, lex_liwc_auxverb, lex_liwc_adverb, lex_liwc_conj, lex_liwc_negate, lex_liwc_verb, lex_liwc_adj, lex_liwc_compare, lex_liwc_interrog, lex_liwc_number, lex_liwc_quant, lex_liwc_affect, lex_liwc_posemo, lex_liwc_negemo, lex_liwc_anx, lex_liwc_anger, lex_liwc_sad, lex_liwc_social, lex_liwc_family, lex_liwc_friend, lex_liwc_female, lex_liwc_male, lex_liwc_cogproc, lex_liwc_insight, lex_liwc_cause, lex_liwc_discrep, lex_liwc_tentat, lex_liwc_certain, lex_liwc_differ, lex_liwc_percept, lex_liwc_see

In [10]:
col_list = ['post_id0', 'text']
data_test = data_test.select([column for column in data_test.columns if column in col_list])

In [11]:
data_test.show(10)

23/02/02 14:48:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text
 Schema: post_id0, text
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/test.csv
+--------+--------------------+
|post_id0|                text|
+--------+--------------------+
|     415|For anxiety/depre...|
|    2691|His response was ...|
|    1322|He would call me ...|
|    2272|Back ground, me a...|
|    1967|Sally is around t...|
|     354|Eat probiotic foo...|
|     965|I obsess about th...|
|    1600|Hi! This is my fi...|
|     527|already a few mon...|
|    1771|Hi everyone, I'm ...|
+--------+--------------------+
only showing top 10 rows



In [12]:
import gensim.parsing.preprocessing as gsp
from pyspark.sql.types import StringType
from gensim import utils


filters = [
           gsp.strip_tags, 
           gsp.strip_punctuation,
           gsp.strip_multiple_whitespaces,
           gsp.strip_numeric,
           gsp.remove_stopwords, 
           gsp.strip_short, 
           gsp.stem_text
          ]

def clean_text(x):
    s = x[1]
    s = s.lower()
    s = utils.to_unicode(s)
    for f in filters:
        s = f(s)
    return (x[0],s,x[2])

In [13]:
input_rdd = data.rdd.map(lambda x : clean_text(x))

input_df  = input_rdd.toDF(['post_id0', 'text', 'label'])
input_df .show(10)

23/02/02 14:48:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


23/02/02 14:48:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


+--------+--------------------+-----+
|post_id0|                text|label|
+--------+--------------------+-----+
|    2092|hello invit compl...|    0|
|    1567|want know stori g...|    0|
|    1550|request accommod ...|    0|
|    2785|reaction meltdown...|    1|
|    1936|throwawai embarra...|    1|
|    1125|enjoi want break ...|    1|
|     945|turn month start ...|    0|
|    1811|start work youtub...|    0|
|    2671|met gui month ago...|    0|
|    1463|feel nake unprote...|    1|
+--------+--------------------+-----+
only showing top 10 rows



In [14]:
def test_clean_text(x):
    s = x[1]
    s = s.lower()
    s = utils.to_unicode(s)
    for f in filters:
        s = f(s)
    return (x[0],s)


input_rdd_test = data_test.rdd.map(lambda x : test_clean_text(x))

test_input_df  = input_rdd_test.toDF(['post_id0', 'text'])
test_input_df .show(10)

23/02/02 14:48:14 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text
 Schema: post_id0, text
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/test.csv


23/02/02 14:48:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text
 Schema: post_id0, text
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/test.csv


+--------+--------------------+
|post_id0|                text|
+--------+--------------------+
|     415|anxieti depress f...|
|    2691|respons constantl...|
|    1322|peepe ask stop ye...|
|    2272|ground girl homel...|
|    1967|salli ag assum ca...|
|     354|eat probiot food ...|
|     965|obsess head try s...|
|    1600|time post reddit ...|
|     527|month advic impro...|
|    1771|studi univers mar...|
+--------+--------------------+
only showing top 10 rows



In [15]:

train_df, test_df = input_df.randomSplit([0.8, 0.2])

In [16]:
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline

tokenizer = Tokenizer(inputCol="text",outputCol="words")
w2v = Word2Vec(vectorSize=300, minCount=0, inputCol="words", outputCol="features")

doc2vec_pipeline = Pipeline(stages=[tokenizer,w2v])

doc2vec_model = doc2vec_pipeline.fit(train_df)

train_df = doc2vec_model.transform(train_df)
test_df = doc2vec_model.transform(test_df)
test_input_df = doc2vec_model.transform(test_input_df)

print ("few rows from test df")
test_df.show(5)
test_input_df.show(3)

23/02/02 14:48:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


23/02/02 14:48:33 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


few rows from test df
23/02/02 14:48:39 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


+--------+--------------------+-----+--------------------+--------------------+
|post_id0|                text|label|               words|            features|
+--------+--------------------+-----+--------------------+--------------------+
|       1|hei assist sure r...|    0|[hei, assist, sur...|[-0.0112574446829...|
|       3|met new boyfriend...|    1|[met, new, boyfri...|[-0.0204435657807...|
|      10|week i’ll fly fam...|    1|[week, i’ll, fly,...|[-0.0129004844896...|
|      14|past coupl month ...|    1|[past, coupl, mon...|[-0.0240485020286...|
|      19|plan fanci month ...|    0|[plan, fanci, mon...|[-0.0139560910769...|
+--------+--------------------+-----+--------------------+--------------------+
only showing top 5 rows

23/02/02 14:48:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text
 Schema: post_id0, text
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/test.csv
+----

In [17]:
#from pyspark.ml.feature import HashingTF, IDF, Tokenizer
#from pyspark.ml import Pipeline


#tokenizer = Tokenizer(inputCol="text", outputCol="words")
#hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(),outputCol="rawFeatures", numFeatures=30)
#idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="features",minDocFreq=5)

#pipeline = Pipeline(stages=[tokenizer, hashingTF, idf])

# Fit the pipeline to training documents.
#model1 = pipeline.fit(train_df)

#train_df = model1.transform(train_df)
#test_df = model1.transform(test_df)
#test_input_df = model1.transform(test_input_df)

#print ("few rows from test df")
#test_input_df.show(3)

In [18]:
## Create initial Logistic regression model
    
lr = LogisticRegression(labelCol="label", featuresCol="features")
model = lr.fit(train_df)

predictions = model.transform(test_df)
evaluator = BinaryClassificationEvaluator(labelCol="label")
accuracy = evaluator.evaluate(predictions)

print ("Accuracy of Logistic Regression Classifier : %g" % accuracy)


test_predictions1 = model.transform(test_input_df)

##test_predictions_result = test_predictions.toDF(['post_id0', 'prediction'])
test_predictions1.show(2)

23/02/02 14:48:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


23/02/02 14:48:48 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


23/02/02 14:48:49 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/02/02 14:48:49 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/02/02 14:48:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


Accuracy of Logistic Regression Classifier : 0.74307
23/02/02 14:49:01 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text
 Schema: post_id0, text
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/test.csv
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|post_id0|                text|               words|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|     415|anxieti depress f...|[anxieti, depress...|[-0.0153171726698...|[-1.3613478523062...|[0.20402132829399...|       1.0|
|    2691|respons constantl...|[respons, constan...|[-0.0147585999086...|[0.21544999667137...|[0.55365510951985...|       0.0|
+--------+--------------------+--------------------+----------

In [19]:

lr = LogisticRegression(labelCol="label", featuresCol="features")
pipeline = Pipeline(stages= [lr])
paramGrid = (ParamGridBuilder()
                     .addGrid(lr.regParam, [0.01, 0.5, 2.0])
                     .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
                     .addGrid(lr.maxIter, [1, 5, 10])
                     .build())
evaluator = BinaryClassificationEvaluator(labelCol="label")
crossValidator = CrossValidator(estimator=pipeline,
                                        evaluator=evaluator,
                                        estimatorParamMaps=paramGrid,
                                        numFolds=5)
# use the Logistic Regression Classifier to train (fit) the model
# and Get the best Logistic Regression model

cv = crossValidator.fit(train_df)
best_model = cv.bestModel.stages[0]

prediction = best_model.transform(test_df)
acc = evaluator.evaluate(prediction)
prediction_test2 = best_model.transform(test_input_df)

print ("The test's accuracy with cross validation = %g" % acc)   

prediction_test2.show(2)

23/02/02 14:49:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


23/02/02 14:49:09 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


23/02/02 14:49:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


23/02/02 14:49:37 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


23/02/02 14:49:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


...

23/02/02 14:50:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


23/02/02 14:51:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


23/02/02 14:51:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


23/02/02 14:51:09 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


The test's accuracy with cross validation = 0.707239
23/02/02 14:51:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text
 Schema: post_id0, text
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/test.csv
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|post_id0|                text|               words|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|     415|anxieti depress f...|[anxieti, depress...|[-0.0153171726698...|[-0.9794271094613...|[0.27300547245274...|       1.0|
|    2691|respons constantl...|[respons, constan...|[-0.0147585999086...|[0.71673674081260...|[0.67188802080208...|       0.0|
+--------+--------------------+--------------------+----------

In [20]:
from pyspark.ml.classification import RandomForestClassifier


rf = RandomForestClassifier(labelCol="label", featuresCol="features")
pipeline = Pipeline(stages=[rf])
# Create ParamGrid for Cross Validation
paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [5, 10]) \
    .addGrid(rf.maxBins, [25, 31]) \
    .addGrid(rf.minInfoGain, [0.01, 0.001]) \
    .addGrid(rf.numTrees, [20, 60]) \
    .addGrid(rf.impurity, ["gini", "entropy"]) \
    .build()
evaluator = BinaryClassificationEvaluator(labelCol="label")
crossValidator = CrossValidator(estimator=pipeline,
                                    evaluator=evaluator,
                                    estimatorParamMaps=paramGrid,
                                    numFolds=10)
# use the Random Forest Classifier to train (fit) the model
cv = crossValidator.fit(train_df)
# and Get the best Random Forest model
best_model = cv.bestModel.stages[0]

prediction = best_model.transform(test_df)
metric = evaluator.evaluate(prediction)
prediction_test3 = best_model.transform(test_input_df)

print ("The metric of test's accuracy= %g" % metric)    
prediction_test3.show(2)

23/02/02 14:51:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


23/02/02 14:51:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


23/02/02 14:52:04 WARN DAGScheduler: Broadcasting large task binary with size 1084.2 KiB
23/02/02 14:52:05 WARN DAGScheduler: Broadcasting large task binary with size 1276.7 KiB
23/02/02 14:52:09 WARN DAGScheduler: Broadcasting large task binary with size 1001.4 KiB
23/02/02 14:52:09 WARN DAGScheduler: Broadcasting large task binary with size 1254.7 KiB
23/02/02 14:52:10 WARN DAGScheduler: Broadcasting large task binary with size 1515.9 KiB
23/02/02 14:52:11 WARN DAGScheduler: Broadcasting large task binary with size 1093.8 KiB
23/02/02 14:52:20 WARN DAGScheduler: Broadcasting large task binary with size 1200.1 KiB
23/02/02 14:52:21 WARN DAGScheduler: Broadcasting large task binary with size 1549.0 KiB
23/02/02 14:52:21 WARN DAGScheduler: Broadcasting large task binary with size 1897.7 KiB
23/02/02 14:52:22 WARN DAGScheduler: Broadcasting large task binary with size 1354.4 KiB
23/02/02 14:52:26 WARN DAGScheduler: Broadcasting large task binary with size 1016.6 KiB
23/02/02 14:52:26 WAR

23/02/02 14:53:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


23/02/02 14:53:51 WARN DAGScheduler: Broadcasting large task binary with size 1148.8 KiB
23/02/02 14:53:55 WARN DAGScheduler: Broadcasting large task binary with size 1219.8 KiB
23/02/02 14:53:56 WARN DAGScheduler: Broadcasting large task binary with size 1472.7 KiB
23/02/02 14:53:57 WARN DAGScheduler: Broadcasting large task binary with size 1063.7 KiB
23/02/02 14:54:06 WARN DAGScheduler: Broadcasting large task binary with size 1181.8 KiB
23/02/02 14:54:07 WARN DAGScheduler: Broadcasting large task binary with size 1523.3 KiB
23/02/02 14:54:08 WARN DAGScheduler: Broadcasting large task binary with size 1875.1 KiB
23/02/02 14:54:09 WARN DAGScheduler: Broadcasting large task binary with size 1342.0 KiB
23/02/02 14:54:12 WARN DAGScheduler: Broadcasting large task binary with size 1021.0 KiB
23/02/02 14:54:12 WARN DAGScheduler: Broadcasting large task binary with size 1288.3 KiB
23/02/02 14:54:13 WARN DAGScheduler: Broadcasting large task binary with size 1564.6 KiB
23/02/02 14:54:14 WAR

...

23/02/02 15:08:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


23/02/02 15:09:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


23/02/02 15:09:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


23/02/02 15:09:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text, label
 Schema: post_id0, text, label
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/train.csv


The metric of test's accuracy= 0.671409
23/02/02 15:09:05 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text
 Schema: post_id0, text
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/test.csv
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|post_id0|                text|               words|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|     415|anxieti depress f...|[anxieti, depress...|[-0.0153171726698...|[28.9411910800106...|[0.48235318466684...|       1.0|
|    2691|respons constantl...|[respons, constan...|[-0.0147585999086...|[32.2331753956451...|[0.53721958992741...|       0.0|
+--------+--------------------+--------------------+--------------------+--

In [21]:
from pyspark.sql.functions import col
test_predictions1 = test_predictions1.select(col("post_id0"), col("prediction"))

In [22]:

test_predictions1.write.csv('prop_prediction_file1.csv')

23/02/02 15:09:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: post_id, text
 Schema: post_id0, text
Expected: post_id0 but found: post_id
CSV file: file:///kaggle/input/soaicommunitydatathon23-stress-detection/test.csv
